# 3.a. Comprehensive Student Performance Analysis Using Pandas
#### Problem Statement:

Perform a thorough analysis of student performance across multiple subjects while handling data issues such as missing marks, duplicate entries, and inconsistent formatting. Use various Pandas functionalities to clean, organize, and derive insights from the data.

#### Objective Scenario:

A college stores internal assessment scores of students in an Excel/CSV file for three subjects: Math, Physics, and Chemistry. However, the dataset contains:

•	Missing marks due to absent students,
•	Duplicate entries for some students,
•	Inconsistent string formatting in names,
•	No unique ID assigned to students.

The academic head wants to:

1.	Clean and prepare the data.
2.	Analyze individual and subject-wise performance.
3.	Identify high-performing students and subjects with the most variability.

#### Dataset:

The data is stored in a CSV file named student_scores.csv, and contains the following columns:

Example data:

Name,Math,Physics,Chemistry

Alice,78,85,

Bob,82,,88

Charlie,75,79,91

Alice,78,85,

David,,92,87

Eve,85,84,88

Bob,82,,88

#### Tasks to be Performed:

🔹 Data Loading and Inspection

•	Load the dataset using Pandas from the CSV file.
•	Display basic information about the dataset using .info() and .describe().
•	Count the number of missing entries in each subject column.
•	Detect and display duplicate rows based on student names.

🔹 Data Cleaning

•	Remove duplicate entries while keeping only the first occurrence of each student.
•	Replace missing subject scores with the median of that subject using .fillna().
•	Assign a new unique identifier column StudentID starting from 1.

🔹 Data Transformation and Normalization

•	Add a new column TotalMarks which sums the marks of all three subjects for each student.
•	Create another column AverageMarks as total marks divided by the number of subjects.
•	Apply min-max normalization on all three subject scores to scale them between 0 and 1, and store them in new columns: Math_Norm, Physics_Norm, Chemistry_Norm.

🔹 Statistical and Performance Analysis

•	Calculate the mean, median, and standard deviation for each subject using normalized scores.
•	Identify the subject with the highest standard deviation (i.e., the most variable scores).
•	Identify the top 3 students based on their average marks.
•	Display subject-wise average performance using a pivot table.



In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

data = {'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'David', 'Eve', 'Bob'],
        'Math': [78, 82, 75, 78, np.nan, 85, 82],
        'Physics': [85, np.nan, 79, 85, 92, 84, np.nan],
        'Chemistry': [np.nan, 88, 91, np.nan, 87, 88, 88]}
pd.DataFrame(data).to_csv('student_scores.csv', index=False)

In [2]:
df = pd.read_csv('student_scores.csv')
print("Dataset Info:"); print(df.info())
print("\nStatistics:"); print(df.describe())
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nDuplicates: {df.duplicated().sum()}")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       7 non-null      object 
 1   Math       6 non-null      float64
 2   Physics    5 non-null      float64
 3   Chemistry  5 non-null      float64
dtypes: float64(3), object(1)
memory usage: 356.0+ bytes
None

Statistics:
           Math    Physics  Chemistry
count   6.00000   5.000000   5.000000
mean   80.00000  85.000000  88.400000
std     3.63318   4.636809   1.516575
min    75.00000  79.000000  87.000000
25%    78.00000  84.000000  88.000000
50%    80.00000  85.000000  88.000000
75%    82.00000  85.000000  88.000000
max    85.00000  92.000000  91.000000

Missing values:
Name         0
Math         1
Physics      2
Chemistry    2
dtype: int64

Duplicates: 2


In [3]:
df = df.drop_duplicates(subset=['Name'], keep='first').reset_index(drop=True)
df = df.fillna(df.median(numeric_only=True))
df['StudentID'] = range(1, len(df) + 1)
print(f"\nCleaned data:\n{df}")


Cleaned data:
      Name  Math  Physics  Chemistry  StudentID
0    Alice  78.0     85.0       88.0          1
1      Bob  82.0     84.5       88.0          2
2  Charlie  75.0     79.0       91.0          3
3    David  80.0     92.0       87.0          4
4      Eve  85.0     84.0       88.0          5


In [4]:
df['TotalMarks'] = df[['Math', 'Physics', 'Chemistry']].sum(axis=1)
df['AverageMarks'] = df['TotalMarks'] / 3

scaler = MinMaxScaler()
for subject in ['Math', 'Physics', 'Chemistry']:
    df[f'{subject}_Norm'] = scaler.fit_transform(df[[subject]])

In [5]:
norm_cols = ['Math_Norm', 'Physics_Norm', 'Chemistry_Norm']
stats = df[norm_cols].agg(['mean', 'median', 'std'])
print(f"\nNormalized Stats:\n{stats.round(4)}")

max_var_subject = stats.loc['std'].idxmax()
print(f"\nHighest variability: {max_var_subject} (std: {stats.loc['std', max_var_subject]:.4f})")

top_students = df.nlargest(3, 'AverageMarks')[['Name', 'AverageMarks']]
print(f"\nTop 3 Students:\n{top_students}")

subject_avg = df[['Math', 'Physics', 'Chemistry']].mean()
print(f"\nSubject Averages:\n{subject_avg.round(2)}")

df.to_csv('student_scores_cleaned.csv', index=False)
print("\nAnalysis complete! Cleaned data saved.")


Normalized Stats:
        Math_Norm  Physics_Norm  Chemistry_Norm
mean       0.5000        0.4538          0.3500
median     0.5000        0.4231          0.2500
std        0.3808        0.3571          0.3791

Highest variability: Math_Norm (std: 0.3808)

Top 3 Students:
    Name  AverageMarks
3  David     86.333333
4    Eve     85.666667
1    Bob     84.833333

Subject Averages:
Math         80.0
Physics      84.9
Chemistry    88.4
dtype: float64

Analysis complete! Cleaned data saved.


# Exercise 3.b: Movie Ratings Aggregation and Review Analysis
#### Problem Statement:
Analyze movie ratings and review text data to find average ratings, detect duplicates, and extract insights using string operations.

#### Objective Scenario:
A streaming platform wants to analyze user reviews. The dataset contains user IDs, movie titles, ratings, and review comments. Some reviews are duplicated and some comments contain extra whitespace or punctuation noise.

#### Dataset:

Stored in movie_reviews.csv with the columns:

Sample Data:

UserID,MovieTitle,Rating,ReviewText

U001,Inception,4.5,"Amazing movie!"

U002,Inception,4.0,"  brilliant  "

U001,Inception,4.5,"Amazing movie!"

U003,Interstellar,5.0,"Mind-blowing!"

U004,Interstellar,, "Outstanding visuals."

U005,Inception,4.2,"Good plot but confusing"

#### Tasks to be Performed:

🔹 Cleaning and Preprocessing

•	Remove duplicate reviews based on UserID and MovieTitle.
•	Handle missing ratings by filling with average rating per movie.
•	Strip and lowercase all review text using vectorized string operations.

🔹 Aggregation

•	Group by MovieTitle and calculate mean, median, and count of ratings.
•	Identify the top-rated movie.

🔹 Text Analysis

•	Count how many times the word “amazing” appears in all reviews.
•	Filter reviews where rating is above 4.0 and review contains the word “plot”.


In [6]:
import pandas as pd
import numpy as np

data = {'UserID': ['U001', 'U002', 'U001', 'U003', 'U004', 'U005'],
        'MovieTitle': ['Inception', 'Inception', 'Inception', 'Interstellar', 'Interstellar', 'Inception'],
        'Rating': [4.5, 4.0, 4.5, 5.0, np.nan, 4.2],
        'ReviewText': ['"Amazing movie!"', '" brilliant "', '"Amazing movie!"', '"Mind-blowing!"', '"Outstanding visuals."', '"Good plot but confusing"']}
pd.DataFrame(data).to_csv('movie_reviews.csv', index=False)

In [7]:
df = pd.read_csv('movie_reviews.csv')
print("Original data:"); print(df)

Original data:
  UserID    MovieTitle  Rating                 ReviewText
0   U001     Inception     4.5           "Amazing movie!"
1   U002     Inception     4.0              " brilliant "
2   U001     Inception     4.5           "Amazing movie!"
3   U003  Interstellar     5.0            "Mind-blowing!"
4   U004  Interstellar     NaN     "Outstanding visuals."
5   U005     Inception     4.2  "Good plot but confusing"


In [8]:
df = df.drop_duplicates(subset=['UserID', 'MovieTitle']).reset_index(drop=True)

df['Rating'] = df.groupby('MovieTitle')['Rating'].transform(lambda x: x.fillna(x.mean()))

df['ReviewText'] = df['ReviewText'].str.strip().str.lower().str.replace('"', '', regex=False)
print(f"\nCleaned data:\n{df}")


Cleaned data:
  UserID    MovieTitle  Rating               ReviewText
0   U001     Inception     4.5           amazing movie!
1   U002     Inception     4.0               brilliant 
2   U003  Interstellar     5.0            mind-blowing!
3   U004  Interstellar     5.0     outstanding visuals.
4   U005     Inception     4.2  good plot but confusing


In [9]:
movie_stats = df.groupby('MovieTitle')['Rating'].agg(['mean', 'median', 'count']).round(2)
print(f"\nMovie Statistics:\n{movie_stats}")

top_movie = movie_stats['mean'].idxmax()
print(f"\nTop-rated movie: {top_movie} (avg: {movie_stats.loc[top_movie, 'mean']})")


Movie Statistics:
              mean  median  count
MovieTitle                       
Inception     4.23     4.2      3
Interstellar  5.00     5.0      2

Top-rated movie: Interstellar (avg: 5.0)


In [12]:
amazing_count = df['ReviewText'].str.count('amazing').sum()
print(f"\n'Amazing' appears {amazing_count} times")

high_rated_plot = df[(df['Rating'] > 4.0) & (df['ReviewText'].str.contains('plot', na=False))]
print(f"\nHigh-rated reviews with 'plot':\n{high_rated_plot[['MovieTitle', 'Rating', 'ReviewText']]}")


'Amazing' appears 1 times

High-rated reviews with 'plot':
  MovieTitle  Rating               ReviewText
4  Inception     4.2  good plot but confusing
